## 1. 패키지 설치, 데이터셋 경로 설정

In [ ]:
# deep-text-recognition-benchmark의 dependency 설치
# !pip install lmdb pillow torchvision nltk natsort fire

In [ ]:
# 대회 목적에 맞게 수정한 ocr 클론
# !git clone https://github.com/mhseo10/customocr.git

### 1-1. 데이터셋 압축 해제 경로 지정
* 라벨 데이터: `./labels/[dataset_name]/ ... `
* 원천 데이터: `./images/[dataset_name]/ ... `
* dataset_name:
  * 한국어 글자체 데이터셋: `'tiw'`
  * 야외 실제 촬영 한글 데이터셋 
    * train: `'hub_train'`
    * validation: `'hub_valid'`
  * 데이콘 데이터셋
    * train: `'train'`
    * test: `'test'`

In [ ]:
hub_timg = './images/hub_train/'
hub_tlabel = './labels/hub_train/' 

hub_vimg = './images/hub_valid/'
hub_vlabel = './labels/hub_valid/'

train_img = './images/train/'
train_label = './labels/train/'

tiw_img = './images/tiw/'
tiw_label = './labels/tiw/'

test_img = './images/test/'

hub_tcrop = './images/crop_train/'
hub_vcrop = './images/crop_valid/'
tiw_crop = './images/crop_tiw/'

lmdb_gt = './lmdb_gt/'

## 2. AI허브 데이터셋 전처리

### 2-1. [한국어 글자체 이미지](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=81)

In [ ]:
import os
import glob
import json
import natsort
import pandas as pd

from tqdm.auto import tqdm, trange
from PIL import Image
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

### 파일 경로, 이미지, 라벨 추출

In [ ]:
data = json.load(open(tiw_label + 'textinthewild_data_info.json', encoding='utf8'))  # json 경로로 변경

In [ ]:
data.__len__(), data.keys()

In [ ]:
img_path = dict()
img_path_li = []
no_img = []

for idx, info in tqdm(enumerate(data['images']), total=len(data['images'])):
    img_path[data['images'][idx]["id"]] = data['images'][idx]["file_name"]
    img_path_li.append(data['images'][idx]["file_name"][:-4])

img_path.__len__()

In [ ]:
file_path_li = []

for (path, dir, files) in os.walk(tiw_img):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        index_number = filename.split('.')[0]
        file_path_li.append(index_number)

file_path_li.__len__()

In [ ]:
# image 파일과 json 파일의 개수가 다르기 때문에 (data, label) 쌍이 맞지 않는 데이터 제거

liset = set(file_path_li)
li2set = set(img_path_li)
black_list = set(liset | li2set) - set(liset & li2set)
black_list.__len__()

In [ ]:
path = []
text = []
bbox = []

for idx, info in tqdm(enumerate(data['annotations']), total=len(data['annotations'])):

    # label 결측치 제거
    if data['annotations'][idx]['text'] == None:
        continue
    if img_path[data['annotations'][idx]['image_id']][:-4] in black_list:
        continue

    # 좌표 이상치 제거
    if data['annotations'][idx]['bbox'][2] + data['annotations'][idx]['bbox'][0] <= data['annotations'][idx]['bbox'][0]:
        continue
    if data['annotations'][idx]['bbox'][3] + data['annotations'][idx]['bbox'][1] <= data['annotations'][idx]['bbox'][1]:
        continue

    # 데이터 추가
    path.append(data['annotations'][idx]['image_id'])
    text.append(data['annotations'][idx]['text'])
    bbox.append(data['annotations'][idx]['bbox'])

len(path), len(text), len(bbox)

### 데이터 전처리(Crop)

In [ ]:
# json의 저장 순서는 (x, y, width, height)
# PIL의 Image.crop()과 순서는 동일하지만, (width, height)를 크기가 아닌 좌표로 변환하는 작업 필요

for i in trange(len(bbox)):
    bbox[i][2] = bbox[i][0] + bbox[i][2]
    bbox[i][3] = bbox[i][1] + bbox[i][3]

In [ ]:
alpha = 0.5  # 이미지 resize 비율
starts = 0  # 오류로 중단하기 전 마지막 인덱스

for idx, i in tqdm(enumerate(bbox[starts:]), total=len(bbox[starts:])):
    idx += starts
    image = Image.open(tiw_img + img_path[path[idx]])
    crop_image = image.crop(i)

    # crop_image.resize((crop_image.size[0] * alpha, crop_image.size[1] * alpha))  # resize
    crop_image.save(tiw_crop + f'train_data_{idx}.jpg')

### 데이터 저장

In [ ]:
image_files = list(glob.glob(tiw_crop + '*'))
image_files = natsort.natsorted(image_files)  # 크롭된 이미지 리스트 정렬

Image.open(image_files[1])

In [ ]:
csv_new_crop = pd.DataFrame([])
csv_new_crop['path'] = image_files
csv_new_crop['text'] = text

In [ ]:
csv_new_crop.info()

In [ ]:
csv_new_crop

In [ ]:
# lmdb 데이터 생성을 위한 gt 파일 저장
csv_new_crop.to_csv(lmdb_gt + 'crop_data.txt', sep='\t', index=False, header=False)

### 2-2. [야외 실제 촬영 한글 이미지](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=105)

### 파일 경로, 이미지, 라벨 추출

In [ ]:
def number_of_subfiles(data_dir):
    s = 0
    _path = []
    _list = []


    for (path, dir, files) in os.walk(data_dir):
        for filename in files:
            ext = os.path.splitext(filename)[-1]

            if ext != '.zip':
                index_number = filename.split('.')[0]
                _list.append(index_number)
                _path.append(path + '/' + filename)
                s += 1
        
    _list.sort()

    return s, _list, _path

In [ ]:
cnt_img, ti_path, train_img = number_of_subfiles(hub_timg)
cnt_label, tl_path, train_label = number_of_subfiles(hub_tlabel)

cnt_img, cnt_label

In [ ]:
liset = set(ti_path)
li2set = set(tl_path)
black_list = set(liset | li2set) - set(liset & li2set)
black_list

In [ ]:
for b in black_list:
    for p in train_label:
        if p.split('.')[-2].split('/')[-1] == b:
            train_label.remove(p)

len(train_label)

In [ ]:
train_img[:50], train_label[:50]

In [ ]:
cnt_img, vi_path, valid_img = number_of_subfiles(hub_vimg)
cnt_label, vl_path, valid_label = number_of_subfiles(hub_vlabel)

cnt_img, cnt_label

In [ ]:
valid_img[:50], valid_label[:50]

### 이미지, 라벨 리스트 생성

In [ ]:
train_json_list = []
valid_json_list = []

for label in tqdm(train_label):
    with open(label, encoding='utf8') as f:
        train_json_list.append(json.load(f))

for label in tqdm(valid_label):
    with open(label, encoding='utf8') as f:
        valid_json_list.append(json.load(f))

train_json_list.__len__(), valid_json_list.__len__()

### 데이터 전처리(Crop)

In [ ]:
def mk_crop(json_list):
    idxs = []    # 파일 순서 인덱스
    lotate = []  # 이미지 좌표
    label = []   # 라벨

    for idx, i in tqdm(enumerate(json_list), total=len(json_list)):
        for jdx, j in enumerate(i['annotations']):

            # 좌표 이상치 제거
            if j['bbox'] == [None, None, None, None]:
                continue

            # 라벨 결측치 제거
            elif j['text'] == "xxx":
                continue

            idxs.append(idx)
            lotate.append(j['bbox'])
            label.append(j['text'])

    return idxs, lotate, label

In [ ]:
t_idxs, t_lotate, t_label = mk_crop(train_json_list)
v_idxs, v_lotate, v_label = mk_crop(valid_json_list)

In [ ]:
# crop을 위한 좌표 정보 수정
for i in trange(len(t_lotate)):
    t_lotate[i][2] = t_lotate[i][0] + t_lotate[i][2]
    t_lotate[i][3] = t_lotate[i][1] + t_lotate[i][3]

for i in trange(len(v_lotate)):
    v_lotate[i][2] = v_lotate[i][0] + v_lotate[i][2]
    v_lotate[i][3] = v_lotate[i][1] + v_lotate[i][3]

### train data 저장

In [ ]:
alpha = 0.5  # 이미지 resize 비율
starts = 0  # 오류로 중단하기 전 마지막 인덱스

for idx, i in tqdm(enumerate(t_lotate), total=len(t_lotate)):
    idx += starts
    image = Image.open(train_img[t_idxs[idx]])
    crop_image = image.crop(i)

    # crop_image.resize((crop_image.size[0] * alpha, crop_image.size[1] * alpha))  # resize
    crop_image.save(hub_tcrop + f'train_data_{str(idx).zfill(7)}.jpg')

paths = sorted(glob.glob(hub_tcrop + '/t*.*'))

data = pd.DataFrame()
data['path'] = paths
data['label'] = t_label
data.to_csv(lmdb_gt + 'hub_tlabel.csv', sep='\t', encoding='utf8', index=False)


### valid data 저장

In [ ]:
alpha = 0.5
starts = 0

for idx, i in tqdm(enumerate(v_lotate), total=len(v_lotate)):
    idx += starts
    image = Image.open(valid_img[v_idxs[idx]])
    crop_image = image.crop(i)

    # crop_image.resize((crop_image.size[0] * alpha, crop_image.size[1] * alpha))  # resize
    crop_image.save(hub_vcrop + f'valid_data_{str(idx).zfill(7)}.jpg')



In [ ]:
data = pd.DataFrame(v_label)
data.to_csv(lmdb_gt + 'hub_vlabel.csv', sep='\t', encoding='utf8', index=False)

label_df = pd.read_csv(lmdb_gt + 'hub_vlabel.csv')
label_df.columns = ['label']
new_data = {'label': 'xxx'}

In [ ]:
# 라벨 데이터가 누락된 행 제거
idx = 54104
temp1 = label_df[label_df.index < idx]
temp2 = label_df[label_df.index >= idx]
label_df = temp1.append(new_data, ignore_index=True).append(temp2, ignore_index=True)
label_df.shape

In [ ]:
paths = sorted(glob.glob(hub_vcrop + '/v*.*'))

label_df['path'] = paths

label_columns = label_df['label']
label_path = label_df['path']

result_df = pd.DataFrame()
result_df['path'] = label_path
result_df['label'] = label_columns
result_df.to_csv('hub_vlabel.csv', sep='\t', encoding='utf8', index=False)

## 3. Dacon 데이터 전처리 및 lmdb 데이터 생성

### 3-1. train.csv 수정

In [ ]:
# 라벨링이 잘못되어있거나 누락된 데이터 수정 -> ./open/train_edit.csv 로 저장
train_csv_path = 'open/train_edit.csv'
train_csv = pd.read_csv(train_csv_path)

# lmdb 데이터 생성을 위한 gt 파일 생성
train_csv.to_csv('./lmdb_data/train.txt', sep='\t', header=False, index=False)

# 최종 학습 단계의 모델 저장을 위한 임시 validation gt 파일 생성
train_csv[:2000].to_csv('./lmdb_data/valid.txt', sep='\t', header=False, index=False)

### 3-2. 학습에 사용할 lmdb 데이터 생성

In [ ]:
'''
window 환경에서 작업 시의 lmdb 파일 생성 명령어:
!python ./hallymocr/create_lmdb_dataset.py --inputPath [데이터 root 경로] --gtFile [txt 파일 경로] --outputPath [데이터 저장 경로] --file_size [데이터 총 크기(GB)]

linux 환경에서 작업 시의 lmdb 파일 생성 명령어:
!python3 ./hallymocr/create_lmdb_dataset.py --inputPath [데이터 root 경로] --gtFile [txt 파일 경로] --outputPath [데이터 저장 경로] --file_size [데이터 총 크기(GB)]
'''

!python ./hallymocr/create_lmdb_dataset.py --inputPath ./images/crop_tiw --gtFile ./lmdb_data/crop_data.txt --outputPath ./result/tiw --file_size 100
!python ./hallymocr/create_lmdb_dataset.py --inputPath ./images/crop_train --gtFile ./lmdb_data/hub_tlabel.txt --outputPath ./result/htrain --file_size 200
!python ./hallymocr/create_lmdb_dataset.py --inputPath ./images/crop_valid --gtFile ./lmdb_data/hub_vlabel.txt --outputPath ./result/hvalid --file_size 100
!python ./hallymocr/create_lmdb_dataset.py --inputPath ./open/ --gtFile ./lmdb_data/train.txt --outputPath ./result/train --file_size 10
!python ./hallymocr/create_lmdb_dataset.py --inputPath ./open/ --gtFile ./lmdb_data/valid.txt --outputPath ./result/valid --file_size 10

## 4. OCR 모델 학습

In [ ]:
ko_txt = ''
e = ord('가')

for i in range(11172):
    ko_txt += chr(e + i)

ko_txt += ' '

In [ ]:
import sys
import random
import string
import torch.backends.cudnn as cudnn
import torch.utils.data
import numpy as np

sys.path.append("./hallymocr")
from hallymocr.train import train
from hallymocr.test import test

In [ ]:
# 하이퍼파라미터 설정
opt = {
    'exp_name': None,
    'train_data': './result/',
    'valid_data': './result/train',
    'manualSeed': 1111,
    'workers': 0,
    'batch_size': 32,
    'num_iter': 100000,
    'valInterval': 1000,
    'saved_model': '',

    'FT': False,
    'adam': False,
    'lr': 1,
    'beta1': 0.9,
    'rho': 0.95,
    'eps': 1e-8,
    'grad_clip': 5,
    'baiduCTC': False,
    'select_data': 'tiw-htrain-hvalid',
    'batch_ratio': '0.3-0.4-0.3',
    'total_data_usage_ratio': '1',
    'batch_max_length': 15,

    'imgH': 256,
    'imgW': 256,
    'rgb': False,
    'character': ko_txt,
    'sensitive': False,
    'PAD': False,
    'data_filtering_off': False,
    'Transformation': 'TPS',  # None|TPS
    'FeatureExtraction': 'ResNet',  # VGG|ResNet|RCNN
    'SequenceModeling': 'BiLSTM',  # None|BiLSTM
    'Prediction': 'Attn',  # CTC|Attn
    'num_fiducial': 20,
    'input_channel': 1,
    'output_channel': 512,
    'hidden_size': 256,
}

# 모델 추가 세부사항 설정
if not opt['exp_name']:
    opt['exp_name'] = '{Transformation}-{FeatureExtraction}-{SequenceModeling}-{Prediction}'.format(**opt)
    opt['exp_name'] += '-Seed{manualSeed}'.format(**opt)
    # print(opt.exp_name)

os.makedirs('./saved_models/{exp_name}'.format(**opt), exist_ok=True)

""" Seed and GPU setting """
random.seed(opt['manualSeed'])
np.random.seed(opt['manualSeed'])
torch.manual_seed(opt['manualSeed'])
torch.cuda.manual_seed(opt['manualSeed'])

cudnn.benchmark = True
cudnn.deterministic = True
opt['num_gpu'] = torch.cuda.device_count()

if opt['num_gpu'] > 1:
    print('------ Use multi-GPU setting ------')
    print('if you stuck too long time with multi-GPU setting, try to set --workers 0')
    opt['workers'] = opt['workers'] * opt['num_gpu']
    opt['batch_size'] = opt['batch_size'] * opt['num_gpu']

### 모델 훈련

In [ ]:
# train
train(opt)

### Dacon 데이터로 2차 학습

In [ ]:
opt['exp_name'] = '{Transformation}-{FeatureExtraction}-{SequenceModeling}-{Prediction}-Seed{manualSeed}'.format(**opt)
opt['saved_model'] = 'saved_models/{Transformation}-{FeatureExtraction}-{SequenceModeling}-{Prediction}-Seed{manualSeed}/best_accuracy.pth'.format(**opt)
opt['num_iter'] = 50000
opt['lr'] = 0.001
opt['select_data'] = 'train'
opt['batch_ratio'] = '1'

In [ ]:
train(opt)

## 5. 최종 결과

### 모델 예측

In [ ]:
# 저장된 모델 load
opt['saved_model'] = 'saved_models/{Transformation}-{FeatureExtraction}-{SequenceModeling}-{Prediction}-Seed{manualSeed}/best_accuracy.pth'.format(**opt)
opt['test_data'] = './open/test'

In [ ]:
result = test(opt)

### CSV 파일 생성

In [ ]:
result_csv_path = 'open/sample_submission.csv'
result_csv = pd.read_csv(result_csv_path)

result_csv['text'] = result

In [ ]:
from datetime import datetime

time = datetime.now().strftime('%m%d_%H%M')
csv_name = f'{time}.csv'
result_csv.to_csv(csv_name, index=False, encoding='utf8')

In [ ]:
pd.read_csv(csv_name)